使用 ResNet-50 在 Imagenette 数据集上进行图像分类训练

## 📋 目录
1. [环境设置](#1-环境设置)
2. [加载数据集](#2-加载数据集)
3. [加载模型](#3-加载模型)
4. [数据预处理](#4-数据预处理)
5. [配置训练参数](#5-配置训练参数)
6. [开始训练](#6-开始训练)
7. [模型评估](#7-模型评估)
8. [结果可视化](#8-结果可视化)

---

## 1. 环境设置

### 1.1 导入必要的库

In [ ]:
from datasets import load_from_disk
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer,
)
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score
import json
from datetime import datetime
from tqdm import tqdm

print("✅ 所有库导入成功!")

### 1.2 配置环境

In [ ]:
# 本地资源路径
CACHE_DIR = "./cache"
DATASET_PATH = os.path.join(CACHE_DIR, "datasets", "imagenette")
MODEL_PATH = os.path.join(CACHE_DIR, "models", "resnet-50")
RESULTS_DIR = "./results"

# 创建结果目录
os.makedirs(RESULTS_DIR, exist_ok=True)

# 设置离线模式
os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

# 设置绘图风格（使用英文避免中文字体问题）
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial', 'Liberation Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

# 启用 matplotlib 内联显示
%matplotlib inline

print("✅ 环境配置完成!")
print(f"📁 数据集路径: {DATASET_PATH}")
print(f"🤖 模型路径: {MODEL_PATH}")
print(f"💾 结果保存目录: {RESULTS_DIR}")

### 1.3 辅助函数

In [ ]:
def print_header(text):
    """打印美化的标题"""
    print("\n" + "=" * 70)
    print(f"  {text}")
    print("=" * 70)

def print_info(text, emoji="ℹ️"):
    """打印提示信息"""
    print(f"{emoji}  {text}")

def print_success(text):
    """打印成功信息"""
    print(f"✅ {text}")

def print_warning(text):
    """打印警告信息"""
    print(f"⚠️  {text}")

print("✅ 辅助函数定义完成!")

---
## 2. 加载数据集

加载本地缓存的 Imagenette 数据集

In [ ]:
print_header("📊 加载数据集")

# 检查数据集是否存在
if not os.path.exists(DATASET_PATH):
    print_warning("数据集未找到! 请先运行: python download_resources.py")
    raise FileNotFoundError(f"数据集不存在: {DATASET_PATH}")

# 加载数据集
dataset = load_from_disk(DATASET_PATH)

# 显示数据集信息
print_info("数据集结构:", "📦")
print(dataset)

# 统计信息
print("\n📈 数据集统计:")
for split in dataset.keys():
    print(f"  • {split:12s}: {len(dataset[split]):,} 样本")

print_success("数据集加载完成!")

### 2.1 查看样本示例

In [ ]:
# 显示几个样本图像
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Dataset Sample Images', fontsize=16, fontweight='bold')

for idx, ax in enumerate(axes.flat):
    sample = dataset['train'][idx]
    ax.imshow(sample['image'])
    ax.set_title(f"Label: {sample['label']}", fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("✅ 样本图像展示完成")

---
## 3. 加载模型

加载预训练的 ResNet-50 模型和图像处理器

### 3.1 加载图像处理器

In [ ]:
print_header("🤖 加载模型和处理器")

# 检查模型是否存在
if not os.path.exists(MODEL_PATH):
    print_warning("模型未找到! 请先运行: python download_resources.py")
    raise FileNotFoundError(f"模型不存在: {MODEL_PATH}")

# 加载图像处理器
print_info("正在加载图像处理器...", "⚙️")
image_processor = AutoImageProcessor.from_pretrained(MODEL_PATH, local_files_only=True)
print_success("图像处理器加载完成")

### 3.2 获取标签信息

In [ ]:
# 获取标签信息
labels = dataset["train"].features["label"].names
num_labels = len(labels)

label2id = {label: str(i) for i, label in enumerate(labels)}
id2label = {str(i): label for i, label in enumerate(labels)}

print_info(f"类别数量: {num_labels}", "🏷️")
print_info(f"类别列表:", "📝")
for i, label in enumerate(labels):
    print(f"  {i}: {label}")

### 3.3 加载预训练模型

In [ ]:
print_info("正在加载预训练模型 ResNet-50...", "🧠")

model = AutoModelForImageClassification.from_pretrained(
    MODEL_PATH,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    local_files_only=True,
)

# 统计模型参数
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print_info(f"模型总参数: {total_params:,}", "📊")
print_info(f"可训练参数: {trainable_params:,}", "🎯")
print_success("模型加载完成!")

---
## 4. 数据预处理

对图像进行预处理(调整大小、归一化等)

In [ ]:
print_header("🎨 数据预处理")

def preprocess_function(examples):
    """数据预处理函数"""
    images = [img.convert("RGB") for img in examples["image"]]
    inputs = image_processor(images)
    inputs["labels"] = examples["label"]
    return inputs

# 预处理数据集
prepared_ds = {}
for split in dataset.keys():
    print_info(f"处理 {split} 集...", "⚙️")
    prepared_ds[split] = dataset[split].map(
        preprocess_function,
        batched=True,
        remove_columns=["image"],
        desc=f"预处理 {split} 集"
    )
    prepared_ds[split].set_format("torch")
    print_success(f"{split} 集预处理完成 ({len(prepared_ds[split])} 样本)")

print_success("所有数据预处理完成!")

---
## 5. 配置训练参数

设置训练配置(批次大小、学习率、轮数等)

In [ ]:
print_header("⚙️ 配置训练参数")

# 训练配置 - 可以根据需要调整
train_batch_size = 32
eval_batch_size = 32
num_epochs = 5
learning_rate = 5e-5

print_info(f"批次大小 (训练): {train_batch_size}", "📦")
print_info(f"批次大小 (评估): {eval_batch_size}", "📦")
print_info(f"训练轮数: {num_epochs}", "🔄")
print_info(f"学习率: {learning_rate}", "📈")

# 计算训练步数
total_steps = (len(prepared_ds["train"]) // train_batch_size) * num_epochs
print_info(f"预计训练步数: {total_steps:,}", "⏱️")

# 定义评估指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# 创建训练参数
training_args = TrainingArguments(
    output_dir="./resnet50-imagenette",
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    eval_strategy="no",  # 训练过程中不进行验证
    save_strategy="epoch",
    logging_steps=50,
    remove_unused_columns=False,
    logging_dir=os.path.join(RESULTS_DIR, "logs"),
    report_to="none",
)

print_success("训练参数配置完成!")

---
## 6. 开始训练

初始化 Trainer 并开始训练模型

> ⚠️ **注意**: 训练可能需要较长时间(取决于硬件配置)

In [ ]:
print_header("🚀 开始训练")

# 初始化 Trainer（
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_ds["train"],
    compute_metrics=compute_metrics,
)

print_success("Trainer 初始化完成")

# 记录开始时间
start_time = datetime.now()
print_info(f"训练开始时间: {start_time.strftime('%Y-%m-%d %H:%M:%S')}", "⏰")

# 开始训练
train_result = trainer.train()

# 记录结束时间
end_time = datetime.now()
training_duration = end_time - start_time
print_success(f"训练完成! 用时: {training_duration}")

# 显示训练指标
train_metrics = train_result.metrics
print("\n📊 训练指标:")
for key, value in train_metrics.items():
    if isinstance(value, float):
        print(f"  • {key}: {value:.4f}")
    else:
        print(f"  • {key}: {value}")

---
## 7. 模型评估

在验证集和测试集上评估模型性能

In [ ]:
print_header("📋 模型评估")

# 在验证集上评估
print_info("在验证集上评估...", "🔍")
eval_results = trainer.evaluate(prepared_ds["validation"])

print("\n📊 验证集指标:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  • {key}: {value:.4f}")
    else:
        print(f"  • {key}: {value}")

print_success("评估完成!")

### 7.1 生成预测结果

In [ ]:
# 生成预测用于可视化
print_info("生成预测结果...", "🎯")

eval_dataset = prepared_ds["validation"]
predictions = trainer.predict(eval_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

print_success(f"预测完成! 共 {len(pred_labels)} 个样本")

---
## 8. 结果可视化

生成多维度的可视化图表

### 8.1 混淆矩阵

In [ ]:
# 混淆矩阵
cm = confusion_matrix(true_labels, pred_labels)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("✅ 混淆矩阵绘制完成")

### 8.2 各类别准确率

In [ ]:
# 计算每个类别的准确率
class_accuracy = []
for i in range(len(labels)):
    mask = true_labels == i
    if mask.sum() > 0:
        acc = (pred_labels[mask] == true_labels[mask]).mean()
        class_accuracy.append(acc * 100)
    else:
        class_accuracy.append(0)

plt.figure(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0, 1, len(labels)))
bars = plt.barh(labels, class_accuracy, color=colors)
plt.xlabel('Accuracy (%)', fontsize=12)
plt.title('Per-Class Accuracy', fontsize=14, fontweight='bold')
plt.xlim(0, 100)

# 添加数值标签
for i, (bar, acc) in enumerate(zip(bars, class_accuracy)):
    plt.text(acc + 1, i, f'{acc:.1f}%', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("✅ 类别准确率图表绘制完成")

### 8.3 整体性能指标

In [ ]:
# 计算各项指标
precision = precision_score(true_labels, pred_labels, average='weighted') * 100
recall = recall_score(true_labels, pred_labels, average='weighted') * 100
f1 = f1_score(true_labels, pred_labels, average='weighted') * 100
accuracy = eval_results['eval_accuracy'] * 100

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
values = [accuracy, precision, recall, f1]
colors_metrics = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

plt.figure(figsize=(10, 6))
bars = plt.bar(metrics, values, color=colors_metrics, alpha=0.8, edgecolor='black', linewidth=1.5)
plt.ylabel('Score (%)', fontsize=12)
plt.title('Overall Performance Metrics', fontsize=14, fontweight='bold')
plt.ylim(0, 100)
plt.grid(axis='y', alpha=0.3, linestyle='--')

# 添加数值标签
for bar, value in zip(bars, values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{value:.2f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ 性能指标图表绘制完成")

### 8.4 真实 vs 预测分布

In [ ]:
# 统计分布
true_counts = np.bincount(true_labels, minlength=len(labels))
pred_counts = np.bincount(pred_labels, minlength=len(labels))

x = np.arange(len(labels))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, true_counts, width, label='True Distribution', color='skyblue', alpha=0.8)
plt.bar(x + width/2, pred_counts, width, label='Predicted Distribution', color='lightcoral', alpha=0.8)

plt.xlabel('Class', fontsize=12)
plt.ylabel('Sample Count', fontsize=12)
plt.title('True vs Predicted Distribution', fontsize=14, fontweight='bold')
plt.xticks(x, labels, rotation=45, ha='right')
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("✅ 分布对比图表绘制完成")

### 8.5 详细分类报告

In [ ]:
# 生成分类报告
print("\n" + "=" * 70)
print("详细分类报告")
print("=" * 70)
report = classification_report(true_labels, pred_labels, target_names=labels, digits=4)
print(report)
print("=" * 70)

print(f"\n📊 整体准确率: {accuracy:.2f}%")
print(f"📊 加权精确率: {precision:.2f}%")
print(f"📊 加权召回率: {recall:.2f}%")
print(f"📊 加权F1分数: {f1:.2f}%")

### 8.6 保存完整可视化图表

In [ ]:
# 创建 2x2 综合图表
fig = plt.figure(figsize=(16, 12))

# 1. 混淆矩阵
ax1 = plt.subplot(2, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right')

# 2. 各类别准确率
ax2 = plt.subplot(2, 2, 2)
bars = plt.barh(labels, class_accuracy, color=plt.cm.viridis(np.linspace(0, 1, len(labels))))
plt.xlabel('Accuracy (%)', fontsize=12)
plt.title('Per-Class Accuracy', fontsize=14, fontweight='bold', pad=20)
plt.xlim(0, 100)
for i, (bar, acc) in enumerate(zip(bars, class_accuracy)):
    plt.text(acc + 1, i, f'{acc:.1f}%', va='center', fontsize=10)

# 3. 性能指标
ax3 = plt.subplot(2, 2, 3)
bars = plt.bar(metrics, values, color=colors_metrics, alpha=0.8, edgecolor='black', linewidth=1.5)
plt.ylabel('Score (%)', fontsize=12)
plt.title('Overall Performance Metrics', fontsize=14, fontweight='bold', pad=20)
plt.ylim(0, 100)
plt.grid(axis='y', alpha=0.3, linestyle='--')
for bar, value in zip(bars, values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{value:.2f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

# 4. 分布对比
ax4 = plt.subplot(2, 2, 4)
plt.bar(x - width/2, true_counts, width, label='True Distribution', color='skyblue', alpha=0.8)
plt.bar(x + width/2, pred_counts, width, label='Predicted Distribution', color='lightcoral', alpha=0.8)
plt.xlabel('Class', fontsize=12)
plt.ylabel('Sample Count', fontsize=12)
plt.title('True vs Predicted Distribution', fontsize=14, fontweight='bold', pad=20)
plt.xticks(x, labels, rotation=45, ha='right')
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()

# 保存图表
plot_file = os.path.join(RESULTS_DIR, 'evaluation_results.png')
plt.savefig(plot_file, dpi=300, bbox_inches='tight')
plt.show()

print_success(f"综合图表已保存到: {plot_file}")

### 8.7 保存结果到文件

In [ ]:
# 保存 JSON 结果
results_summary = {
    "training_duration": str(training_duration),
    "start_time": start_time.strftime('%Y-%m-%d %H:%M:%S'),
    "end_time": end_time.strftime('%Y-%m-%d %H:%M:%S'),
    "train_metrics": {k: float(v) if isinstance(v, (int, float, np.number)) else str(v) 
                     for k, v in train_metrics.items()},
    "validation_metrics": {k: float(v) if isinstance(v, (int, float, np.number)) else str(v) 
                          for k, v in eval_results.items()},
    "num_labels": num_labels,
    "labels": labels,
}

results_file = os.path.join(RESULTS_DIR, "training_results.json")
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)

print_success(f"JSON 结果已保存到: {results_file}")

# 保存分类报告
report_file = os.path.join(RESULTS_DIR, 'classification_report.txt')
with open(report_file, 'w', encoding='utf-8') as f:
    f.write("Image Classification Report\n")
    f.write("=" * 70 + "\n\n")
    f.write(report)
    f.write("\n\n" + "=" * 70 + "\n")
    f.write(f"Overall Accuracy: {accuracy:.2f}%\n")
    f.write(f"Weighted Precision: {precision:.2f}%\n")
    f.write(f"Weighted Recall: {recall:.2f}%\n")
    f.write(f"Weighted F1-Score: {f1:.2f}%\n")

print_success(f"分类报告已保存到: {report_file}")
print_info(f"所有结果保存在: {os.path.abspath(RESULTS_DIR)}", "📁")

---
## 🎉 训练完成!

### 📊 结果总结

- ✅ 训练完成
- ✅ 模型评估完成
- ✅ 可视化图表生成
- ✅ 结果文件保存

### 📁 输出文件

所有结果保存在 `results/` 目录:
- `evaluation_results.png` - 综合可视化图表
- `classification_report.txt` - 详细分类报告
- `training_results.json` - JSON 格式结果

### 🚀 下一步

- 尝试调整超参数提升性能
- 使用更大的数据集
- 尝试其他预训练模型
- 部署模型进行推理